In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from databaseemulator import DataBaseEmulator
from domain_model import StateVariable, Activity, ActivityCategory, Scenario, Actor, \
    ActorCategory, Sinusoidal, EgoVehicle, State, Tag, DetectedActivity, Linear
from stats import KDE
from cutin_detection import TARGET, STAT, EGO_CATEGORY
%matplotlib inline

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_scenarios", "cut_in_scenarios.json")
cutins = DataBaseEmulator(filename)

# Delete a few items for which we know they are not really cut ins.
for i in [39, 130, 258, 288]:
    cutins.delete_item("scenario", i)
    
nscenarios = len(cutins.collections["scenario"])
print("Number of scenarios: {:d}".format(nscenarios))

# Step 1: Item generation

An item denotes a combination of tags. In this case study, we consider tags for the longitudinal activities and tags for the lateral activities. For the longitudinal activities, we have:

- A: Accelerating
- C: Cruising
- D: Decelerating

For the lateral activities, we have:

- F: Lane following
- L: Lane changing

When combining these different tags, there are six possibilities: AF, CF, DF, AL, CL, and DL.

Our goal is to determine the so-called transition probabilities: what is the probability to go from one item to another item. To calculate this probability, use is made of the Markov assumption. This means that it is assumed that the probability of going from item $X_i$ to item $X_{i+1}$ only depends on item $X_i$ and not on any previous items.

In [ ]:
# Go through all cut-ins and store the sequence of longitudinal activities.
activities = []
for i in range(nscenarios):
    # Get the acts in chronological order.
    scenario = cutins.get_ordered_item("scenario", i)
    acts = [scenario.acts[i] for i in np.argsort([act[2] for act in scenario.acts])]
    
    # List the longitudinal activities of the target vehicle.
    curr_activities = []
    for act in acts:
        if act[0].name == "target vehicle" and \
                act[1].activity_category.state == StateVariable.LON_TARGET:
            curr_activities.append(act[1].name[0])
    activities.append(curr_activities)

In [ ]:
# Store in a dictionary the possibilities of 'item' sequences.
# 'af' means "accelerating and lane following"
# 'dl' means "decelerating and lane changing"
# 'cf', 'df', 'al', and 'cl' are similarly defined.
def get_transition(activities, main_act, other_acts):
    counter = np.zeros(1+len(other_acts), dtype=np.int)
    for activity in activities:
        try:
            i = activity.index(main_act)
        except ValueError:
            pass
        else:
            if i == len(activity)-1:
                counter[len(other_acts)] += 1
                continue
            for j, other_act in enumerate(other_acts):
                if activity[i+1] == other_act:
                    counter[j] += 1
    counter = counter / np.sum(counter)
    probs = dict()
    probs['end'] = counter[-1]
    for i, other_act in enumerate(other_acts):
        probs[other_act+'l'] = counter[i]
    return probs
transition = dict(af=dict(al=1),
                  cf=dict(cl=1),
                  df=dict(dl=1),
                  al=get_transition(activities, 'a', ['c', 'd']),
                  cl=get_transition(activities, 'c', ['a', 'd']),
                  dl=get_transition(activities, 'd', ['a', 'c']))

In [ ]:
acts, counts = np.unique([activity[0] for activity in activities], return_counts=True)
transition['start'] = dict()
for act, count in zip(acts, counts/np.sum(counts)):
    transition['start'][act+'f'] = count
transition

The dictionary above shows the transition probabilities. We always start at "start". From the "start", we go through either of the items AF, CF, or DF. Once we are at item - let's say - AF, we can look what the probabilities are for the next possible items. In the case of AF, we go with probability 1 to AL. This is by the definition of the scenario. From item AL, we can either go to item CL or DL or we can end the scenario.

The next code is used to randomly generate sequence of items based on the above defined transition probabilities. It also calculates the probability of a given sequence. 

In [ ]:
def next_item(probs):
    """ It is important that the probabilities in probs add up to 1. """
    rand = np.random.rand()
    for key, value in probs.items():
        if rand <= value:
            return key
        rand -= value

def generate_sequence(transitions, seed=None):
    if seed is not None:
        np.random.seed(0)
    sequence = ['start']
    while not sequence[-1] == 'end':
        sequence.append(next_item(transitions[sequence[-1]]))
    return sequence[1:-1]

def probability_sequence(transitions, sequence):
    sequence = ['start'] + sequence + ['end']
    prob = 1
    for act1, act2 in zip(sequence[:-1], sequence[1:]):
        prob *= transitions[act1][act2]
    return prob

In [ ]:
np.random.seed(0)
n = 10000
seqs = [generate_sequence(transition) for _ in range(n)]

In [ ]:
sequences = [['cf', 'cl'],
             ['af', 'al'],
             ['df', 'dl'],
             ['cf', 'cl', 'al'],
             ['cf', 'cl', 'dl'],
             ['af', 'al', 'cl'],
             ['af', 'al', 'dl'],
             ['df', 'dl', 'al'],
             ['df', 'dl', 'cl']] #,
x=[             ['cf', 'cl', 'al', 'cl'],
             ['cf', 'cl', 'al', 'dl'],
             ['cf', 'cl', 'dl', 'cl'],
             ['cf', 'cl', 'dl', 'al'],
             ['af', 'al', 'cl', 'al'],
             ['af', 'al', 'cl', 'dl'],
             ['af', 'al', 'dl', 'al'],
             ['af', 'al', 'dl', 'cl'],
             ['df', 'dl', 'al', 'cl'],
             ['df', 'dl', 'al', 'dl'],
             ['df', 'dl', 'cl', 'al'],
             ['df', 'dl', 'cl', 'dl']]
probs = [probability_sequence(transition, seq) for seq in sequences]
# .count() is very slow like this, but fine for now...
mc_probs = [seqs.count(seq)/n for seq in sequences]
real_seqs = [[y[0]+'f']+[x+'l' for x in y] for y in activities]
real_probs = [real_seqs.count(seq)/len(real_seqs) for seq in sequences]
for seq, prob, mc_prob, real_prob in zip(sequences, probs, mc_probs, real_probs):
    print("{:13s} {:.4f} {:.4f} {:.4f}".format(' '.join(seq), prob, mc_prob, real_prob))
print()
print("{:13s} {:.4f} {:.4f} {:.4f}".format("Total", np.sum(probs), np.sum(mc_probs), 
                                           np.sum(real_probs)))

Each columns has the following meaning:

1. The corresponding sequence of items.
2. The probability of this sequence based on the transition probabilities and the Markov assumption.
3. The number this sequence is generated divided over the total number of generated sequences.
4. The probability of this sequence based on the real data.

The similarity between the second and third column demonstrate that sequences are generated according to the calculated probability. Still, these probabilities are significantly different from the real probabilities (fourth column). The inaccuracy is introduced by the Markov assumption. 

# Step 2: Parameter generation

For each of the activities accelerating, cruising, decelerating, and lane changing, parameters are determined. Furthermore, some initial parameters are determined. For each of these parameters, a Kernel Density Estimation (KDE) is used to estimate the probability density. These KDEs are used to generate the parameters.

The following parameters are considered:

- Accelerating: Initial speed, mean acceleration, speed difference
- Cruising: Initial speed, log(duration), speed difference
- Decelerating: Initial speed, mean acceleration, speed difference
- Changing lane: Initial lateral position, Final lateral position, and duration

Note: For cruising, the log of the duration is used, because then the values are better behaved for constructing the KDE.

In [ ]:
# Functions for obtaining the important parameters.
def parm_acc_dec(activity: Activity):
    """ Get initial speed, mean acceleration, and speed difference. """
    vstart, vend = activity.get_state(time=[activity.tstart, activity.tend])[0]
    amean = (vend - vstart) / (activity.tend - activity.tstart)
    return vstart, amean, vend-vstart

def parm_cruise(activity: Activity):
    """ Get initial speed, log(duration), and speed difference. """
    vstart, vend = activity.get_state(time=[activity.tstart, activity.tend])[0]
    return vstart, np.log(activity.tend-activity.tstart), vend-vstart

def parm_lc(activity: Activity):
    """ Get initial position, end position, and duration. """
    ystart, yend = activity.get_state(time=[activity.tstart, activity.tend])
    return ystart, yend, activity.tend-activity.tstart

In [ ]:
# Get the parameters of the longitudinal activities.
par_acc = []
par_cru = []
par_dec = []
par_lc = []
for i in range(len(cutins.collections["activity"])):
    activity = cutins.get_item("activity", i)
    if activity.name == "acceleration target":
        par_acc.append(parm_acc_dec(activity))
    elif activity.name == "deceleration target":
        par_dec.append(parm_acc_dec(activity))
    elif activity.name == "cruising target":
        par_cru.append(parm_cruise(activity))
    elif activity.name in ["right lane change", "left lane change"]:
        par_lc.append(parm_lc(activity))

In [ ]:
par_initx = np.zeros(nscenarios)
par_egov = np.zeros(nscenarios)
par_tarv = np.zeros(nscenarios)
for i in range(nscenarios):
    scenario = cutins.get_ordered_item("scenario", i)
    for actor in scenario.actors:
        if actor.name == "ego vehicle":
            break
    par_egov[i] = scenario.get_state(actor, StateVariable.SPEED, scenario.time["start"])
    
    for actor in scenario.actors:
        if actor.name == "target vehicle":
            break
    par_tarv[i], par_initx[i] = scenario.get_state(actor, StateVariable.LON_TARGET, 
                                                   scenario.time["start"])

The following code is used to generate a scenario. It is a rather naive approach in the sense that it is assumed that the parameters of each of the activities are independent of all other parameters.

In [ ]:
class Approach1:
    def __init__(self):
        # Create KDEs of initial parameters.
        self.kde_egov = KDE(par_egov)
        self.kde_initx = KDE(par_initx)
        
        # Create KDE of individual lane changes parameters.
        self.kde_lc_ystart = KDE(np.array(par_lc)[:, 0])
        self.kde_lc_yend = KDE(np.array(par_lc)[:, 1])
        self.kde_lc_dt = KDE(np.array(par_lc)[:, 2])
        
        # Create KDE of each of the longitudinal activities.
        self.kde_a = KDE(np.array(par_acc))
        self.kde_c = KDE(np.array(par_cru))
        self.kde_d = KDE(np.array(par_dec))
        
        # Compute the bandwidths
        for kde in (self.kde_egov, self.kde_initx, self.kde_lc_ystart, self.kde_lc_yend,
                    self.kde_lc_dt, self.kde_a, self.kde_c, self.kde_d):
            kde.compute_bandwidth()
    
    def generate(self, items):
        inits = [self.kde_egov.sample()[0][0], self.kde_initx.sample()[0][0],
                 np.random.rand()]
        lc = [self.kde_lc_ystart.sample()[0][0], self.kde_lc_yend.sample()[0][0], 
              self.kde_lc_dt.sample()[0][0]]
        lon = []
        for item in items[1:]:
            if item[0] == "a":
                lon.append(self.kde_a.sample()[0])
            elif item[0] == "c":
                lon.append(self.kde_c.sample()[0])
                lon[-1][1] = np.exp(lon[-1][1])
            else:
                lon.append(self.kde_d.sample()[0])
        return (inits, lc, lon)
A1 = Approach1()

It might be possible that the generated parameters are invalid for the given sequence of items. For example, consider the case that we have the items AF and AL. In that case, it means that the lane change activity ends *before* the acceleration activity ends. So if the parameters are such that the lane change activity ends *after* the acceleration activity ends, the parameters are invalid. 

The following checks are done:

- The initial speed of the ego vehicle should be strictly positive.
- The initial lateral position of the target vehicle should be 2 meters away from the center of the ego lane.
- Each activity should be strictly longer than 0 seconds.
- The mean acceleration and the speed difference for an acceleration activity should be strictly positive.
- The mean acceleration and the speed difference for a deceleration activity should be strictly negative.
- The last longitudinal activity should start before the end of the lane change.
- The last longitudinal activity should end after the end of the lane change.

In [ ]:
def valid_cutin(sequence, parameters, verbose=False):
    # Ego speed should be positive.
    if parameters[0][0] < 0:
        if verbose:
            print("Negative ego speed")
        return False
    
    # Start y should be at least 2 m away from ego lane. End should be less than 1.5 m.
    if np.abs(parameters[1][0]) < 2 or np.abs(parameters[1][1]) > 1.5:
        if verbose:
            print("Wrong lane change parameters")
        return False
    
    durations = np.zeros(len(sequence) - 1)
    for i, (item, par) in enumerate(zip(sequence[1:], parameters[2])):
        # Each longitudinal activity should be longer than 0. 
        # For acc/dec, it means than amean is pos/neg.
        if (item[0] == "a" and par[1] <= 0) or (item[0] == "d" and par[1] >= 0):
            if verbose:
                print("Wrong acceleration/deceleration")
            return False
        if item[0] == "c" and par[1] <= 0:
            if verbose:
                print("Wrong cruising", sequence, parameters)
            return False
        
        # With acceleration/deceleration, there should be a speed increase/decrease.
        if (item[0] == "a" and par[2] <= 0) or (item[0] == "d" and par[2] >= 0):
            return False
        
        if item[0] in ["a", "d"]:
            durations[i] = par[2] / par[1]
        else:
            durations[i] = par[1]
        
    # Last lon activity should start before end lane change and end after it.
    total_duration = -durations[0] * parameters[0][2]
    total_duration += np.sum(durations[:-2])
    if total_duration > parameters[1][2]:
        if verbose:
            print("Longitudinal activities too long")
        return False
    if total_duration + durations[-1] < parameters[1][2]:
        if verbose:
            print("Longitudinal activities too short")
        return False
    
    return True

The next code is an example of a generated "scenario", where the scenario is now presented by a sequence of items and the parameters.

In [ ]:
np.random.seed(4)
sequence = generate_sequence(transition)
parameters = A1.generate(sequence)
while not valid_cutin(sequence, parameters, verbose=True):
    parameters = A1.generate(sequence)
print(sequence)
print(parameters)
valid_cutin(sequence, parameters)

# Step 3: Create Scenario object

Now that we have the code to generate a sequence of items and the corresponding parameters, we can also represent this sequence of items and list of parameters as a Scenario object. The following code converts the sequence of items and the list of parameters to a Scenario object. The following assumptions are used:

- For all activities other than cruising, a sinusoidal model is used.
- For cruising, a linear model is used.
- The scenario happens at a straight road.
- No other actors are involved than the ego vehicle and the target vehicle.

In [ ]:
LC_TARGET = ActivityCategory(Sinusoidal(), StateVariable.LAT_TARGET, 
                             name="lane change target",
                             tags=[Tag.VehicleLateralActivity_ChangingLane])
DEC_TARGET = ActivityCategory(Sinusoidal(), StateVariable.SPEED, name="deceleration target",
                              tags=[Tag.VehicleLongitudinalActivity_DrivingForward_Braking])
ACC_TARGET = ActivityCategory(Sinusoidal(), StateVariable.SPEED, name="acceleration target",
                              tags=[Tag.VehicleLongitudinalActivity_DrivingForward_Accelerating])
CRU_TARGET = ActivityCategory(Linear(), StateVariable.SPEED, name="cruising target",
                              tags=[Tag.VehicleLongitudinalActivity_DrivingForward_Cruising])

def create_scenario(sequence, parameters):
    # Create actors.
    ego = EgoVehicle(EGO_CATEGORY, 
                     initial_states=[State(StateVariable.SPEED, parameters[0][0]),
                                     State(StateVariable.LONGITUDINAL_POSITION, 0)],
                     name="ego vehicle")
    target = Actor(TARGET,
                   initial_states=[State(StateVariable.LONGITUDINAL_POSITION, parameters[0][1])],
                   name="target vehicle")
    
    # Create lane change activity of the target.
    lane_change = DetectedActivity(LC_TARGET, 0, parameters[1][2],
                                   parameters=dict(xstart=parameters[1][0],
                                                   xend=parameters[1][1]))
    
    # Create longitudinal activities of the target.
    lon_activities = []
    tstart = 0
    for item, pars in zip(sequence[1:], parameters[2]):
        if len(lon_activities) == 0:
            # Determine the start time for this one.
            if item[0] == "c":
                tstart = -parameters[0][2] * pars[1]
            else:
                tstart = -parameters[0][2] * pars[2] / pars[1]
        if item[0] == "a":
            lon_activities.append(DetectedActivity(ACC_TARGET, tstart, pars[2]/pars[1],
                                                   parameters=dict(xstart=pars[0],
                                                                   xend=pars[0]+pars[2])))
            tstart += pars[2] / pars[1]
        elif item[0] == "d":
            lon_activities.append(DetectedActivity(DEC_TARGET, tstart, pars[2]/pars[1],
                                                   parameters=dict(xstart=pars[0],
                                                                   xend=pars[0]+pars[2])))
            tstart += pars[2] / pars[1]
        else:
            lon_activities.append(DetectedActivity(CRU_TARGET, tstart, pars[1],
                                                   parameters=dict(xstart=pars[0],
                                                                   xend=pars[0]+pars[2])))
            tstart += pars[1]
        lon_activities[-1].parameters.update(dict(tstart=lon_activities[-1].tstart,
                                                  tend=tstart))
            
    # Create the scenario
    scenario = Scenario(lane_change.tstart, lane_change.tend, STAT)
    scenario.set_activities([lane_change]+lon_activities)
    scenario.set_actors([ego, target])
    scenario.set_acts([(target, activity, activity.tstart) 
                       for activity in [lane_change]+lon_activities])
    return scenario            

s2 = create_scenario(sequence, parameters)
s1 = cutins.get_item("scenario", 0)

# Step 4: Compare two scenarios

Now that we are able to generate a Scenario object, we are able to compare the generated scenario with an original scenario. To do this, we collect "features" of the scenarios. This is a very much a manual process. As an alternative, a more generalizable metric could be used to define the "difference" between two scenarios, but an earlier study showed that it is better to have a more "manual" approach, such that "the expert" can choose what is an important difference and what is not.

In the case of the cut-in scenario, the following "features" are used:

1. The initial longitudinal position of the target vehicle with respect to the ego vehicle.
2. The total duration of the scenario.
3. The lateral position of the target vehicle with respect to the ego vehicle's lane center.
4. The speed of the target vehicle.
5. The initial speed of the ego vehicle.

Note that the first, second, and fifth features are scalars and the third and fourth features are vectors.

In [ ]:
# Define the different 'measures'.
# Measure 1: Initial longitudinal position of target vehicle.
def get_initial_lon_position(scenario: Scenario):
    actor = scenario.get_actor_by_name("target vehicle")
    if actor.initial_states:
        return actor.initial_states[0].value[0]
    return scenario.get_state(actor, StateVariable.LON_TARGET, time=scenario.time["start"])[1]

# Meaure 2: Total duration of the scenario.
def get_duration(scenario: Scenario):
    return scenario.time["end"] - scenario.time["start"]

# Measure 3: Get vector of y positions.
def get_lateral_position(scenario: Scenario):
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LAT_TARGET:
            return activity.get_state(npoints=20)

# Measure 4: Get vector of speeds of target vehicle.
def get_speed_target(scenario: Scenario):
    actor = scenario.get_actor_by_name("target vehicle")
    speed = scenario.get_state(actor, StateVariable.SPEED,
                               np.linspace(scenario.time["start"], scenario.time["end"], 50))
    if speed is not None:
        return speed
    return scenario.get_state(actor, StateVariable.LON_TARGET,
                              np.linspace(scenario.time["start"], scenario.time["end"], 50))[:, 1]

# Measure 5: Get initial speed of ego vehicle.
def get_initial_speed_ego(scenario: Scenario):
    actor = scenario.get_actor_by_name("ego vehicle")
    if actor.initial_states:
        return actor.initial_states[0].value[0]
    return scenario.get_state(actor, StateVariable.SPEED, scenario.time["start"])

In [ ]:
features = [(get_initial_lon_position(scenario),
             get_duration(scenario),
             get_lateral_position(scenario),
             get_speed_target(scenario),
             get_initial_speed_ego(scenario)) for scenario in
            [cutins.get_ordered_item("scenario", i) for i in range(nscenarios)]]

In [ ]:
i, j, k = 0, 0, 1
difference = np.zeros((nscenarios*(nscenarios-1)//2, 5))
for i in range(nscenarios*(nscenarios-1)//2):
    for m in range(5):
        difference[i, m] = np.sqrt(np.mean((features[j][m] - features[k][m])**2))
    k += 1
    if k == nscenarios:
        j += 1
        k = j + 1

In [ ]:
np.std(difference, axis=0)

In [ ]:
features[7]

In [ ]:
s = cutins.get_ordered_item("scenario", 7)

In [ ]:
s.get_state(s.get_actor_by_name("target vehicle"), StateVariable.LAT_TARGET,
            np.linspace(scenario.time["start"], scenario.time["end"], 50))

In [ ]:
s.activities[0].tend

In [ ]:
s.get_actor_by_name("target vehicle").properties